# Preparing data
Prediction of sales is the central task in this challenge. you want to predict daily sales in various stores up to 6 weeks ahead of time. This will help the company plan ahead of time. 

The following steps outline the various sub tasks needed to effectively do this: 


In [1]:
# importing of libraries
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import os,sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from timeseries import TimeSeries
from clean import Clean
sns.set()
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

2022-05-25 04:46:28.004293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 04:46:28.004422: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/martin/Documents/pharm_sales/notebooks


# Preprocessing
It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values and generating new features from already existing features. 

In our case, you have a few datetime columns to preprocess. you can extract the following from them:
weekdays
weekends 
number of days to holidays
Number of days after holiday
Beginning of month, mid month and ending of month
(think of more features to extract), extra marks for it
            
As a final thing, you have to scale the data. This helps with predictions especially when using machine learning algorithms that use Euclidean distances. you can use the standard scaler in sklearn for this.


In [2]:
store = pd.read_csv("../data/store.csv")
df = pd.read_csv("../data/train.csv")
clean_df = Clean(df)
clean_df.merge_df(store,'Store')
clean_df.save(name='../data/unclean_train.csv')
clean_df.drop_missing_values()
clean_df.fix_outliers('Sales',25000)
clean_df.remove_unnamed_cols()
clean_df.transfrom_time_series("Store","Date")
clean_df.save(name="../data/training.csv")


/tmp/ipykernel_8234/2395586539.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train.csv")
2022-05-25 04:46:40,391:logger:Successfully initialized clean class
2022-05-25 04:46:40,774:logger:Successfully merged the dataframe
2022-05-25 04:46:48,947:logger:Successfully saved the dataframe
2022-05-25 04:46:50,517:logger:Successfully dropped the columns with missing values
2022-05-25 04:46:50,532:logger:Successfully stored the features
2022-05-25 04:46:50,592:logger:Successfully handled outliers
2022-05-25 04:46:50,640:logger:Successfully removed columns with head unnamed
2022-05-25 04:46:51,338:logger:Successfully transformed data to time series data
2022-05-25 04:46:57,764:logger:Successfully saved the dataframe


In [3]:
df = pd.read_csv("../data/test.csv")
clean_df = Clean(df)
clean_df.merge_df(store,'Store')
clean_df.save(name='../data/unclean_test.csv')
clean_df.drop_missing_values()
clean_df.fix_outliers('Sales',25000)
clean_df.remove_unnamed_cols()
clean_df.transfrom_time_series("Store","Date")
clean_df.save(name="../data/testing.csv")

2022-05-25 04:46:58,001:logger:Successfully initialized clean class
2022-05-25 04:46:58,020:logger:Successfully merged the dataframe
2022-05-25 04:46:58,343:logger:Successfully saved the dataframe
2022-05-25 04:46:58,393:logger:Successfully dropped the columns with missing values
2022-05-25 04:46:58,394:logger:Successfully stored the features
2022-05-25 04:46:58,396:logger:Successfully handled outliers
2022-05-25 04:46:58,402:logger:Successfully removed columns with head unnamed
2022-05-25 04:46:58,477:logger:Successfully transformed data to time series data
2022-05-25 04:46:58,752:logger:Successfully saved the dataframe


In [4]:
train = pd.read_csv("../data/training.csv")
test = pd.read_csv("../data/testing.csv")

Feature Engineering

In [5]:
daily_sales = clean_df.aggregations(train,'Store','Sales','Open','sum')
daily_customers = clean_df.aggregations(train,'Store','Customers','Open','sum')
avg_sales = clean_df.aggregations(train,'Store','Sales','Open','mean')
avg_customers = clean_df.aggregations(train,'Store','Customers','Open','mean')
train['DailySales'] = train['Store'].map(daily_sales)
train['DailyCustomers'] = train['Store'].map(daily_customers)
train['AvgSales'] = train['Store'].map(avg_sales)
train['AvgCustomers'] = train['Store'].map(avg_customers)

2022-05-25 04:46:59,588:logger:successful aggregation
2022-05-25 04:46:59,612:logger:successful aggregation
2022-05-25 04:46:59,683:logger:successful aggregation
2022-05-25 04:46:59,708:logger:successful aggregation


Label Encoding

In [6]:
clean_df.label_encoding(train,test)

2022-05-25 04:46:59,750:logger:Successfully stored the features
2022-05-25 04:47:00,279:logger:Successfully encoded your categorical data


Scaling Data

In [7]:
training_data_ = train[train.columns.difference(['DayOfWeek','Day', 'Month', 'Year', 'DayOfYear','WeekOfYear'])]
testing_data_ = test[test.columns.difference(['DayOfWeek','Day', 'Month', 'Year', 'DayOfYear','WeekOfYear'])]

In [8]:
train_transformation=clean_df.generate_transformation(training_data_,"numeric","number")
test_transformation=clean_df.generate_transformation(testing_data_,"numeric","number")

2022-05-25 04:47:00,646:logger:Successfully transformed numerical data
2022-05-25 04:47:00,664:logger:Successfully transformed numerical data


In [9]:
indexes = ['DayOfWeek','Day', 'Month', 'Year', 'DayOfYear','WeekOfYear']
train_transformed = pd.DataFrame(train_transformation,columns=train.columns.difference(indexes))
test_transformed = pd.DataFrame(test_transformation,columns=test.columns.difference(indexes))
train_index = train[indexes]
test_index = test[indexes]
train = pd.concat([train_index,train_transformed],axis=1)
test = pd.concat([test_index,test_transformed],axis=1)

In [10]:
train.sort_values(["Year","Month","Day"], ascending=False ,ignore_index=True, inplace=True)
test.sort_values(["Year","Month","Day"], ascending=False ,ignore_index=True, inplace=True)

In [11]:
train.to_csv("../data/cleaned_train.csv",index=False)
test.to_csv("../data/cleaned_test.csv",index=False)